# Updating the SVD

---

In many applications which are based on the SVD, arrival of new data requires SVD of the new matrix. Instead of computing from scratch, existing SVD can be updated.

## Prerequisites

The reader should be familiar with concepts of singular values and singular vectors, related perturbation theory, and algorithms.
 
## Competences 

The reader should be able to recognise applications where SVD updating can be sucessfully applied and apply it.

---

## Facts

For more details see
[M. Gu and S. C. Eisenstat, A Stable and Fast Algorithm for Updating the Singular Value Decomposition][GE93]
and [M. Brand, Fast low-rank modifications of the thin singular value decomposition][Bra06]
and the references therein.

[GE93]: http://www.cs.yale.edu/publications/techreports/tr966.pdf "M. Gu and S. C. Eisenstat, 'A Stable and Fast Algorithm for Updating the Singular Value Decomposition', Tech.report, Yale University, 1993."

[Bra06]: http://www.sciencedirect.com/science/article/pii/S0024379505003812 "M. Brand, 'Fast low-rank modifications of the thin singular value decomposition', Linear Algebra and its Appl, 415 (20-30) 2006."

1. Let $A\in\mathbb{R}^{m\times n}$ with $m\geq n$ and $\mathop{\mathrm{rank}}(A)=n$, and  let $A=U\Sigma V^T$ be its SVD.
   Let $a\in\mathbb{R}^{n}$ be a vector, and let $\tilde A=\begin{bmatrix} A \\ a^T\end{bmatrix}$. Then
   $$\begin{bmatrix} A \\ a^T\end{bmatrix} =\begin{bmatrix} U & \\ & 1 \end{bmatrix} 
   \begin{bmatrix} \Sigma \\ a^TV \end{bmatrix}  V^T.
   $$
   Let $\begin{bmatrix} \Sigma \\ a^T V \end{bmatrix} = \bar U \bar \Sigma \bar V^T$ be the SVD of the half-arrowhead matrix. _This SVD can be computed in $O(n^2)$ operations._ Then 
   $$\begin{bmatrix} A \\ a^T\end{bmatrix} =
   \begin{bmatrix} U & \\ & 1 \end{bmatrix} \bar U \bar\Sigma \bar V^T V^T \equiv
   \tilde U \bar \Sigma \tilde V^T
   $$
   is the SVD of $\tilde A$. 
   
2. Direct computation of $\tilde U$ and $\tilde V$ requires $O(mn^2)$ and $O(n^3)$ operations. However, these multiplications can be performed using Fast Multipole Method. This is not (yet) implemented in Julia and is "not for the timid" (quote by Steven G. Johnson).

3. If $m<n$ and $\mathop{\mathrm{rank}}(A)=n$, then
   $$
   \begin{bmatrix} A \\ a^T\end{bmatrix} =\begin{bmatrix} U & \\ & 1 \end{bmatrix} 
   \begin{bmatrix} \Sigma & 0 \\ a^T V & \beta\end{bmatrix} \begin{bmatrix} V^T \\ v^T \end{bmatrix},
   $$
   where $\beta=\sqrt{\|a\|_2^2-\|V^T a\|_2^2}$ and $v=(I-VV^T)a$. Notice that $V^Tv=0$ by construction.
   Let $\begin{bmatrix} \Sigma & 0 \\ a^T V &  \beta\end{bmatrix} = \bar U \bar \Sigma \bar V^T$ be the SVD of 
   the half-arrowhead matrix. Then 
   $$\begin{bmatrix} A \\ a^T\end{bmatrix} =
   \begin{bmatrix} U & \\ & 1 \end{bmatrix} \bar U \bar\Sigma \bar V^T \begin{bmatrix} V^T \\ v^T \end{bmatrix}
   \equiv \tilde U \bar \Sigma \tilde V^T
   $$
   is the SVD of $\tilde A$.
   
3. Adding a column $a$ to $A$ is equivalent to adding a row $a^T$ to $A^T$.

3. If $\mathop{\mathrm{rank}}(A)<\min\{m,n\}$ or if we are using SVD approximation of rank $r$, and if we want to keep the rank of the approximation (this is the common case in practice), then the formulas in Fact 1 hold approximately. More precisely, the updated rank $r$ approximation is __not__ what we would get by computing the approximation of rank $r$ of the updated matrix, but is sufficient in many applications. 

### Example - Adding row to a tall matrix

If $m>=n$, adding row does not increase the size of $\Sigma$.

In [1]:
using Arrowhead

In [2]:
function mySVDaddrow{T}(svdA::Tuple,a::Vector{T})
    # Create the transposed half-arrowhead
    m,r,n=size(svdA[1],1),length(svdA[2]),size(svdA[3],1)
    b=svdA[3]'*a
    if m>=n || r<m
        M=HalfArrow(svdA[2],b)
    else
        β=sqrt(vecnorm(a)^2-vecnorm(b)^2)
        M=HalfArrow(svdA[2],[b;β])
    end
    tols=[1e2,1e2,1e2,1e2]
    U,σ,V=svd(M,tols)
    # Return the updated SVD
    if m>=n || r<m
        return [svdA[1] zeros(T,m); zeros(T,1,r) one(T)]*V, σ, svdA[3]*U
    else
        # Need one more row of svdA[3] - v is orthogonal projection
        v=a-svdA[3]*b
        v=v/norm(v)
        return [svdA[1] zeros(T,m); zeros(T,1,r) one(T)]*V, σ, [svdA[3] v]*U
    end
end

mySVDaddrow (generic function with 1 method)

In [3]:
A=rand(10,6)
a=rand(6)

6-element Array{Float64,1}:
 0.221399
 0.768015
 0.434068
 0.669569
 0.349026
 0.937966

In [4]:
svdA=svd(A)

(
10x6 Array{Float64,2}:
 -0.243555   0.762451   -0.161994   -0.259465   -0.138777    0.375221 
 -0.246189  -0.0119037   0.498387   -0.0175853  -0.322755    0.195748 
 -0.317533  -0.361891    0.043925   -0.494296   -0.0696735   0.359471 
 -0.178403  -0.184867   -0.427682   -0.0953758   0.473642    0.29752  
 -0.354006   0.178985    0.0179635   0.194722    0.551791    0.0198701
 -0.422596  -0.216432   -0.0906625   0.617458   -0.0657186   0.126564 
 -0.387754   0.118349   -0.130384   -0.309193    0.0544702  -0.753264 
 -0.239722   0.217977    0.578327    0.173827    0.229692   -0.0575153
 -0.395297  -0.325427    0.0996438  -0.229156   -0.118191   -0.101937 
 -0.281834   0.0841711  -0.41303     0.285384   -0.518548   -0.0750654,

[4.0669223594657655,1.1197418382358004,0.9946878371522861,0.8485590415477365,0.674202190337477,0.45382025423254235],
6x6 Array{Float64,2}:
 -0.372081   0.689015   -0.227543   0.0961875   0.392363   -0.414533
 -0.481432  -0.625971   -0.467782  -0.216512    0.31664

In [5]:
typeof(svdA)

Tuple{Array{Float64,2},Array{Float64,1},Array{Float64,2}}

In [6]:
U,σ,V=mySVDaddrow(svdA,a)

Remedy 3 


(
11x6 Array{Float64,2}:
 -0.223588   0.741827   -0.190412    -0.0927312    0.0730524  -0.487543 
 -0.233061   0.0402103   0.501492     0.00365628   0.315254   -0.187775 
 -0.300305  -0.151736    0.102506     0.571258     0.0186496  -0.391427 
 -0.171012  -0.141164   -0.40899      0.179551    -0.47827    -0.205638 
 -0.336224   0.142568   -0.00437316  -0.220858    -0.521894    0.101039 
 -0.407184  -0.283662   -0.112532    -0.347054     0.150506    0.122691 
 -0.362356   0.265978   -0.105999     0.305316    -0.058596    0.617229 
 -0.226749   0.18621     0.552408    -0.270214    -0.208507    0.114133 
 -0.374709  -0.153561    0.142184     0.377957     0.114103    0.0938557
 -0.268033   0.0521039  -0.428354    -0.159883     0.553611    0.120539 
 -0.32008   -0.415477   -0.0411357   -0.359083    -0.068915   -0.298231 ,

[4.286621653219806,1.204402274736068,0.9951041265454403,0.9577533795504506,0.6774785431809869,0.498626262441919],
6x6 Array{Float64,2}:
 -0.349721   0.642482  -0.268356  

In [7]:
# Check the residual and orthogonality
norm([A;a']*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

(5.589014826080515e-15,1.348466742720976e-15,1.1244364582318845e-15)

### Example - Adding row to a flat matrix

In [8]:
# Now flat matrix
A=rand(6,10)
a=rand(10)
svdA=svd(A)

(
6x6 Array{Float64,2}:
 -0.492899  -0.632294    0.219741   -0.040012   0.250835   -0.494419 
 -0.281561   0.498295    0.78851    -0.224182   0.0107329   0.0174794
 -0.463948   0.559196   -0.528127   -0.047077   0.258021   -0.352622 
 -0.53031   -0.138779   -0.188737   -0.31602   -0.689295    0.29815  
 -0.242783  -0.132008   -0.0781563  -0.17855    0.62207     0.706167 
 -0.34979    0.0502126   0.0965287   0.902317  -0.0910671   0.208178 ,

[4.030811316421142,1.1737792968182705,0.9750642580493722,0.8398159144651589,0.5987941180123774,0.33909047741554293],
10x6 Array{Float64,2}:
 -0.315261  -0.275586     0.062729   -0.147609   -0.0134245  -0.0914208
 -0.179688   0.104346     0.569515    0.178287    0.422191    0.0625051
 -0.349289  -0.00768558   0.239263   -0.677574   -0.232717   -0.14997  
 -0.222914  -0.456098     0.186059    0.0425505   0.48718     0.362133 
 -0.409889  -0.227428    -0.12584     0.011468   -0.0518926  -0.03536  
 -0.22917    0.302136    -0.323826    0.138017    0.52

In [9]:
U,σ,V=mySVDaddrow(svdA,a)
norm([A;a']*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

(8.010550988849256e-15,1.6726991807756699e-15,2.9517375933747436e-15)

### Example - Adding columns

This can be viewed as adding rows to the transposed matrix, an elegant one-liner in Julia.

In [10]:
function mySVDaddcol{T}(svdA::Tuple,a::Vector{T})
    reverse(mySVDaddrow(reverse(svdA),a))
end 

mySVDaddcol (generic function with 1 method)

In [11]:
# Tall matrix
A=rand(10,6)
a=rand(10)
svdA=svd(A)
U,σ,V=mySVDaddcol(svdA,a)
norm([A a]*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

Remedy 3 


(3.136901403015872e-15,3.2608700902653993e-15,1.753752094124507e-15)

In [12]:
# Flat matrix
A=rand(6,10)
a=rand(6)
svdA=svd(A)
U,σ,V=mySVDaddcol(svdA,a)
norm([A a]*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

(1.883988810118734e-15,1.0269170176271777e-15,9.120667403637087e-16)

In [13]:
# Square matrix
A=rand(10,10)
a=rand(10);
svdA=svd(A);

In [14]:
U,σ,V=mySVDaddrow(svdA,a)
norm([A;a']*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

Remedy 3 


(3.5713454174155424e-14,1.4196141008832963e-15,6.4889861569114546e-15)

In [15]:
U,σ,V=mySVDaddcol(svdA,a)
norm([A a]*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

Remedy 3 


(3.533535952914256e-14,1.4269971076026336e-15,1.1828442053492742e-15)

### Example - Updating a low rank approximation


In [16]:
# Adding row to a tall matrix
A=rand(10,6)
svdA=svd(A)
a=rand(6)
# Rank of the approximation
r=4

4

In [17]:
svdAr=(svdA[1][:,1:r], svdA[2][1:r],svdA[3][:,1:r])
U,σ,V=mySVDaddrow(svdAr,a)
norm([A;a']-U*diagm(σ)*V'), svdvals([A;a']), σ

(0.42361407360119013,[4.144646535891786,1.339670070904022,1.0412845751261728,0.6682949457350921,0.4236093424586042,0.34306699570834687],[4.143020619171327,1.3375963820720134,1.0408915929849003,0.6612773119109294])

In [18]:
# Adding row to a flat matrix
A=rand(6,10)
svdA=svd(A)
a=rand(10)
# Rank of the approximation
r=4

4

In [19]:
svdAr=(svdA[1][:,1:r], svdA[2][1:r],svdA[3][:,1:r])
U,σ,V=mySVDaddrow(svdAr,a)
norm([A;a']-U*diagm(σ)*V'), svdvals([A;a']), σ

(0.8608726378200711,[4.089986400967123,1.2934152855408454,1.085652181293417,0.7867763810535159,0.6380854088271951,0.5312664041778705,0.2920823286705993],[4.079984631527442,1.279181466529765,0.9800014657665581,0.7828186661537818])